# Constant

In [18]:
import certifi
import json
import time
import random
import psycopg2
import os
from psycopg2.extras import execute_values

API_KEY = "f8fe2458856a3904f2a649ebf1e82a68"
PG_HOST = os.environ.get('POSTGRES_HOST','')
PG_PORT = os.environ.get('POSTGRES_PORT','')
PG_DB = os.environ.get('POSTGRES_DB','')
PG_USER = os.environ.get('POSTGRES_USER','')
PG_PASS = os.environ.get('POSTGRES_PASSWORD','')

# Common library

In [55]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen


def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    time.sleep(1)
    return json.loads(data)

def emptyToNull(val):
    if val:
        return val
    else:
        return None

def connectDB():
    con = psycopg2.connect(user=PG_USER,
                       password=PG_PASS,
                       host=PG_HOST,
                       port=PG_PORT,
                       database=PG_DB)
    return con

# Global variable

In [ ]:
dbCon=connectDB()
dbCon.autocommit = True
dbCur=dbCon.cursor()

# Get symbol list

In [16]:
url = ("https://financialmodelingprep.com/api/v3/stock/list?apikey="+API_KEY)
retDict = get_jsonparsed_data(url)
symbol_dat = []

# Save request quotas by specify range
if retDict:
    for item in retDict[0:3]:
        symbol_dat.append((item.get('symbol'), item.get('name')))
    execute_values(dbCur, "INSERT INTO public.stock (symbol, stockname) VALUES %s", symbol_dat)
#print(symbol_dat)

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


[('SPY', 'SPDR S&P 500 ETF Trust'), ('CMCSA', 'Comcast Corporation'), ('KMI', 'Kinder Morgan, Inc.')]


In [ ]:
  "symbol" : "IBBJ",
  "companyName" : "DEFIANCE NASDAQ JUNIOR BIOTECHNOLOGY ETF ",
  "exchange" : "NASDAQ",
  "ipoDate" : "2020-08-04",
  "delistedDate" : "2022-09-06"

# Get delisted companies

In [34]:
startpage = 0
maxpage = 2
delistedCom_dat = []

for page in range(startpage,  maxpage):
    url = ("https://financialmodelingprep.com/api/v3/delisted-companies?page="+str(page)+"&apikey="+API_KEY)
    retDict=get_jsonparsed_data(url)
    if retDict:
        print("page:", page)
        for item in retDict:
            delistedCom_dat.append((item.get('symbol'), item.get('companyName'), item.get('exchange'), item.get('ipoDate'), item.get('delistedDate')))
        execute_values(dbCur, "INSERT INTO public.delistedcompany (symbolid, companname, exchange, ipodate, delisteddate) VALUES %s", delistedCom_dat)
    else:
        break

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


page: 0


ForeignKeyViolation: insert or update on table "delistedcompany" violates foreign key constraint "fk_stock"
DETAIL:  Key (symbolid)=(IBBJ) is not present in table "stock".


# Get historical dividends

In [57]:
dividend_dat = []
for symbol, symbolName in symbol_dat[0:1]:
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/"+symbol+"?apikey="+API_KEY)
    retDict = get_jsonparsed_data(url)
    if retDict:
        for item in retDict.get('historical'):
            dividend_dat.append((symbol, item.get('date'), item.get('label'), item.get('adjDividend'), item.get('dividend'), emptyToNull(item.get('recordDate')), emptyToNull(item.get('paymentDate')), emptyToNull(item.get('declarationDate'))))
        #print(dividend_dat)
        execute_values(dbCur, "INSERT INTO public.historicaldevidend (symbolid, dividenddate, label, adjdividend, dividend, recorddate, paymentdate, declarationdate) VALUES %s", dividend_dat)
    else:
        break

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


# Clean whatever we need to

In [58]:
dbCur.close()
dbCon.close()